In [1]:
from nemo_text_processing.text_normalization.normalize import Normalizer

In [2]:
normaliser_sv = Normalizer(input_case='cased', lang='sv', cache_dir="/tmp/nemo_cache_sv", overwrite_cache=False, post_process=True)

Created /tmp/nemo_cache_sv/sv_tn_True_deterministic_cased__tokenize.far
Created /tmp/nemo_cache_sv/sv_tn_True_deterministic_verbalizer.far


In [3]:
from pathlib import Path
APIFILES = Path("/home/joregan/rdapi_subset_files")
OUTFILES = Path("/home/joregan/rdapi_subset_norm")

In [4]:
import json

In [14]:
!pip install beautifulsoup4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 26.1 MB/s eta 0:00:00


In [15]:
from bs4 import BeautifulSoup

In [16]:
def get_speaker_paragraphs(html):
    if "<p>" in html or "<P>" in html:
        soup = BeautifulSoup(html, 'html.parser')
        paragraphs = []
        for para in soup.find_all("p"):
            if para.text.strip() != "" and not para.text.strip().startswith("STYLEREF Kantrubrik"):
                paragraphs.append(para.text.strip())
        return paragraphs
    else:
        text = html.strip().replace("\r\n", "\n").replace("\r", "\n")
        return text.split("\n")

In [20]:
with open(str(APIFILES / "H101CU11")) as apifile:
    rdfile = json.load(apifile)

In [21]:
for viddata in rdfile['videodata']:
    for speaker in viddata['speakers']:
        paras = get_speaker_paragraphs(speaker['anftext'])
        normed = normaliser_sv.normalize_list(paras)
        speaker['anftext'] = "\r\n".join(normed)

100%|██████████| 1/1 [00:00<00:00,  5.12it/s]


In [22]:
with open("/tmp/check", "w") as outfile:
    json.dump(rdfile, outfile)